Load packages

In [2]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np
from datetime import date
import ast

Define DA model with Flexibility Options

In [3]:
DAFOmodel = pyo.AbstractModel()

#Sets
DAFOmodel.S = pyo.RangeSet(1, 5) #Set of scenarios
DAFOmodel.G = pyo.RangeSet(1, 5) #Set of generators
DAFOmodel.R = pyo.RangeSet(1, 4) #Set of tiers

#Parameters
DAFOmodel.VC = pyo.Param(DAFOmodel.G) #Variable cost
DAFOmodel.VCUP = pyo.Param(DAFOmodel.G) #Variable cost
DAFOmodel.VCDN = pyo.Param(DAFOmodel.G) #Variable cost
DAFOmodel.CAP = pyo.Param(DAFOmodel.G) #Capacity
DAFOmodel.REDA=pyo.Param(within=pyo.NonNegativeIntegers) #Maximum DA RE
DAFOmodel.DEMAND=pyo.Param(within=pyo.NonNegativeIntegers) #Electricty demand
DAFOmodel.D1=pyo.Param(within=pyo.NonNegativeIntegers) #linear cost coefficient for demand slack variable
DAFOmodel.D2=pyo.Param(within=pyo.NonNegativeIntegers) #quadratic cost coefficient for demand slack variable

#Parameters specific to the FO
DAFOmodel.RR= pyo.Param(DAFOmodel.G) #Ramp rate
DAFOmodel.RE = pyo.Param(DAFOmodel.S) #Renewable generation at each scenario [FO buyer]
DAFOmodel.PEN=pyo.Param(within=pyo.NonNegativeIntegers) #Penalty for inadequate flexibility up [FO buyer]
DAFOmodel.PENDN=pyo.Param(within=pyo.NonNegativeIntegers) #Penalty for inadequate flexibility down [FO buyer]
DAFOmodel.smallM=pyo.Param(within=pyo.NonNegativeReals) #Parameter that helps choose among alternative optima
DAFOmodel.probTU=pyo.Param(DAFOmodel.R) #probability of exercise FO up
DAFOmodel.probTD=pyo.Param(DAFOmodel.R) #probability of exercise FO dn

# Variables
DAFOmodel.d = pyo.Var(domain=pyo.NonNegativeReals) #slack for demand that makes demand elastic and helps us with degeneracy
DAFOmodel.xDA=pyo.Var(DAFOmodel.G, domain=pyo.NonNegativeReals) #DA energy schedule for generators
DAFOmodel.rgDA=pyo.Var(domain=pyo.NonNegativeReals) #DA energy schedule for renewables
DAFOmodel.du= pyo.Var(DAFOmodel.S)

#Variables [FO]
DAFOmodel.hsu  = pyo.Var(DAFOmodel.R, DAFOmodel.G, domain=pyo.NonNegativeReals) #supply FO up
DAFOmodel.hsd  = pyo.Var(DAFOmodel.R, DAFOmodel.G, domain=pyo.NonNegativeReals) #supply FO down
DAFOmodel.hdu  = pyo.Var(DAFOmodel.R, domain=pyo.NonNegativeReals) #demand FO up
DAFOmodel.hdd  = pyo.Var(DAFOmodel.R, domain=pyo.NonNegativeReals)  #demand FO down
DAFOmodel.sdu  = pyo.Var(DAFOmodel.R, domain=pyo.NonNegativeReals)  #Self-supply FO up [FO buyer]
DAFOmodel.sdd  = pyo.Var(DAFOmodel.R, domain=pyo.NonNegativeReals) #Self-supply FO down [FO buyer]
DAFOmodel.y    = pyo.Var(DAFOmodel.S, domain=pyo.NonNegativeReals) #Auxiliary variable



#Define objective function
def obj_expression(m):
    return sum(m.VC[g]*m.xDA[g] for g in m.G)+sum(m.probTU[r]*m.VCUP[g]*m.hsu[r,g] for g in m.G for r in m.R)\
+sum(m.probTU[r]*m.PEN*m.sdu[r]  for r in m.R)-sum(m.probTD[r]*(m.VCDN[g])*m.hsd[r,g] for g in m.G for r in m.R)\
-sum(m.probTD[r]*m.PENDN*m.sdd[r] for r in m.R)+sum(m.y[s] for s in m.S)*m.smallM \
+sum(0.2*m.D1*(m.d+m.du[s])  for s in m.S) +0.2*m.D2*sum((m.d+m.du[s])*(m.d+m.du[s]) for s in m.S  )\
###### create symmetry with demand curve in real-time, #+ m.D2 *m.d*m.d

DAFOmodel.OBJ = pyo.Objective(rule=obj_expression)

#Define constraints - Numbering of constraints follows paper
def DA_energy_balance(model):
    return sum(model.xDA[g] for g in model.G)+ model.rgDA+model.d == model.DEMAND
DAFOmodel.Con3 = pyo.Constraint(expr=DA_energy_balance) 

def DA_flexup_balance(model,r):
    return sum(model.hsu[r,g] for g in model.G) == model.hdu[r]
DAFOmodel.Con4UP = pyo.Constraint(DAFOmodel.R, rule=DA_flexup_balance)

def DA_flexdn_balance(model,r):
    return sum(model.hsd[r,g] for g in model.G) == model.hdd[r]
DAFOmodel.Con4DN = pyo.Constraint(DAFOmodel.R, rule=DA_flexdn_balance)

def DA_flex_demand(model,s):
    return -model.du[s]+sum(model.hdd[r]+model.sdd[r] for r in model.R if r<=s-1 )-sum(model.hdu[r]+model.sdu[r] for r in model.R if r>=s )==model.RE[s]-model.rgDA
DAFOmodel.Con6 = pyo.Constraint(DAFOmodel.S, rule=DA_flex_demand)

def DA_flex_demand_bound(model,s):
    return sum(model.hdd[r]+model.sdd[r] for r in model.R if r<=s-1 )+sum(model.hdu[r]+model.sdu[r] for r in model.R if r>=s )<=model.y[s]
DAFOmodel.Con7 = pyo.Constraint(DAFOmodel.S, rule=DA_flex_demand_bound)

def Y2(model,s):
    return model.y[s]>=model.rgDA-model.RE[s]
DAFOmodel.Con8  = pyo.Constraint(DAFOmodel.S, rule=Y2)  

def Y1(model,s):
    return model.y[s]>=model.RE[s]-model.rgDA
DAFOmodel.Con9 = pyo.Constraint(DAFOmodel.S, rule=Y1)  
    
def RRUP(model,g):
    return sum(model.hsu[r,g] for r in model.R)<=model.RR[g]
DAFOmodel.Con10up = pyo.Constraint(DAFOmodel.S, rule=RRUP)  

def RRDN(model,g):
    return sum(model.hsd[r,g] for r in model.R)<=model.RR[g]
DAFOmodel.Con10dn = pyo.Constraint(DAFOmodel.S, rule=RRDN) 

def DA_capacity_cons(model, g):
    return model.xDA[g] +sum(model.hsu[r,g] for r in model.R)<= model.CAP[g]
DAFOmodel.Con11 = pyo.Constraint(DAFOmodel.G, rule=DA_capacity_cons)

def DA_down_cons(model, g):
    return  sum(model.hsd[r,g] for r in model.R)<= model.xDA[g]
DAFOmodel.Con12= pyo.Constraint(DAFOmodel.G, rule=DA_down_cons)

#Record duals
DAFOmodel.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

Stochastic Program that is used to simulate the impact of DA decisions on RT

In [4]:
RTsim = pyo.AbstractModel()
#Sets
RTsim.S = pyo.RangeSet(1, 5)
RTsim.G = pyo.RangeSet(1, 5)

#Parameters
RTsim.VC = pyo.Param(RTsim.G)
RTsim.VCUP = pyo.Param(RTsim.G)
RTsim.VCDN = pyo.Param(RTsim.G)
RTsim.CAP = pyo.Param(RTsim.G)
RTsim.RR= pyo.Param(RTsim.G)

RTsim.prob=pyo.Param(RTsim.S)
RTsim.RE = pyo.Param(RTsim.S)
RTsim.DEMAND=pyo.Param(within=pyo.NonNegativeIntegers)
RTsim.D1=pyo.Param(within=pyo.NonNegativeIntegers)
RTsim.D2=pyo.Param(within=pyo.NonNegativeIntegers)
RTsim.xDA=pyo.Param(RTsim.G, domain=pyo.NonNegativeReals)
RTsim.REDA=pyo.Param( domain=pyo.NonNegativeReals)
RTsim.PEN=pyo.Param(within=pyo.NonNegativeIntegers)
RTsim.PENDN=pyo.Param()
RTsim.DAdr=pyo.Param()

# Variables
RTsim.xup = pyo.Var(RTsim.S, RTsim.G, domain=pyo.NonNegativeReals)
RTsim.xdn = pyo.Var(RTsim.S, RTsim.G, domain=pyo.NonNegativeReals)
RTsim.d= pyo.Var(RTsim.S)
RTsim.rgup = pyo.Var(RTsim.S, domain=pyo.NonNegativeReals)
RTsim.rgdn = pyo.Var(RTsim.S, domain=pyo.NonNegativeReals)
RTsim.sdup = pyo.Var(RTsim.S, domain=pyo.NonNegativeReals)
RTsim.sddn = pyo.Var(RTsim.S, domain=pyo.NonNegativeReals)

#Objective
def obj_expression(m):
    return sum(m.prob[s]*(m.VCUP[g]*m.xup[s,g]-m.VCDN[g]*m.xdn[s,g]) for g in m.G for s in m.S)+m.PENDN*sum(m.prob[s]*m.sdup[s] for s in m.S)+m.PEN*sum(m.prob[s]*m.sddn[s] for s in m.S)\
+sum(m.prob[s]*(m.D1*(m.DAdr+m.d[s])+m.D2*(m.DAdr+m.d[s])*(m.DAdr+m.d[s])) for s in m.S)\
-sum(m.prob[s]*(m.D1*(m.DAdr)+m.D2*(m.DAdr)*(m.DAdr)) for s in m.S)
RTsim.OBJ = pyo.Objective(rule=obj_expression)

#Constraints
def RT_energy_balance(model,s):
    return sum(model.xup[s,g]-model.xdn[s,g] for g in model.G)+ model.rgup[s]-model.rgdn[s] +model.d[s]== 0

def RT_RE_availability(model,s):
    return model.rgup[s]-model.rgdn[s]+model.sdup[s]-model.sddn[s] ==model.RE[s]-model.REDA 

def RT_ramp_up(model,s,g):
    return model.xup[s,g] <=model.RR[g]

def RT_ramp_dn(model,s,g):
    return model.xdn[s,g] <= model.RR[g]
def RT_capacity_cons(model,s, g):
    return model.xDA[g]+model.xup[s,g] <= model.CAP[g]

def RT_capacity_min(model,s, g):
    return model.xDA[g]-model.xdn[s,g] >=0

# Numbering of constraints follows the RT problem
RTsim.Con3 = pyo.Constraint(RTsim.S, rule=RT_energy_balance)
RTsim.Con4 = pyo.Constraint(RTsim.S, rule= RT_RE_availability)
RTsim.Con5up = pyo.Constraint(RTsim.S,RTsim.G, rule=RT_ramp_up)
RTsim.Con5dn = pyo.Constraint(RTsim.S, RTsim.G,rule=RT_ramp_dn)
RTsim.Con6 = pyo.Constraint(RTsim.S,RTsim.G, rule=RT_capacity_cons)
RTsim.Con7 = pyo.Constraint(RTsim.S,RTsim.G, rule=RT_capacity_min)

#Record duals
RTsim.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

Use CPLEX to solve the problem [quadratic objective function due to elastic demand]

In [5]:
opt = pyo.SolverFactory('cplex',executable="C:/Program Files/IBM/ILOG/CPLEX_Studio2212/cplex/bin/x64_win64/cplex")


Read input data from excel and record outputs in indvidual excel files

In [6]:
input_data=pd.read_csv('data/FO_input_sectionV.csv',index_col=0)
Summary_cost=pd.DataFrame()
Summary_price_convergence=pd.DataFrame()
Summary_premium_convergence_up=pd.DataFrame()
Summary_premium_convergence_down=pd.DataFrame()
Summary_margin_intermittency=pd.DataFrame()
Summary_costD=pd.DataFrame()
Summary_costCURT=pd.DataFrame()
for l in input_data.columns:
        dataDA=input_data[l].to_dict()
        for k in range(0,len(input_data[l])):
            dataDA[input_data.index[k]]=ast.literal_eval(input_data[l][input_data.index[k]])
        dataDA = {None: dataDA}
        i = DAFOmodel.create_instance(dataDA)
        opt.solve(i)

        i.pprint()
        
        #Values to pass from DA to RT module
        Energy=pd.DataFrame()
        E_grid_data = {(G): v.value for (G), v in i.xDA.items()}
        Energy['en']= pd.DataFrame.from_dict(E_grid_data, orient="index", columns=["en"])
        temp=i.rgDA.value
        Total=pd.DataFrame()
        Total.at['cost','DA']=sum(np.multiply(np.array([i.VC[k] for k in i.G]),np.array([i.xDA[g].value for g in i.G])))
        Total.at['price','DA']=i.dual[i.Con3]
        E_grid_data = {(R): v.value for (R), v in i.hdu.items()}
        demand = pd.DataFrame.from_dict(E_grid_data, orient="index", columns=["hdu"])
        E_grid_data = {(R): v.value for (R), v in i.hdd.items()}
        demand ['hdd']= pd.DataFrame.from_dict(E_grid_data, orient="index", columns=["hdd"])
        E_grid_data = {(R,G): v.value for (R,G), v in i.hsu.items()}
        df = pd.DataFrame.from_dict(E_grid_data, orient="index", columns=["hsu"])
        E_grid_data = {(R,G): v.value for (R,G), v in i.hsd.items()}
        df ['hsd']= pd.DataFrame.from_dict(E_grid_data, orient="index", columns=["hsd"])
        df['R']=[df.index[i][0] for i in range(0,len(df))]
        df['G']=[df.index[i][1] for i in range(0,len(df))]
        Prices=pd.DataFrame()
        Prices['up']=[i.dual[i.Con4UP[r]] for r in i.R]
        Prices['down']=[i.dual[i.Con4DN[r]] for r in i.R]
        Gross_margins=pd.DataFrame()
        Gross_margins["en"]=np.multiply(i.dual[i.Con3]-np.array([i.VC[k] for k in i.G]), Energy['en'])
        #Gross_margins.at[0,"en"]=i.rgDA.value*i.dual[i.Con3]
        for tier in range(1,5):
            temp2=np.multiply(df["hsu"].loc[df["R"]==tier].reset_index(drop=True), Prices.at[tier-1,"up"]-np.multiply(np.array([i.VCUP[k] for k in i.G]),i.probTU[tier]))
            temp2.index=range(1,6)
            Gross_margins["up"+str(tier)]=temp2
            temp2=np.multiply(df["hsd"].loc[df["R"]==tier].reset_index(drop=True), Prices.at[tier-1,"down"]+np.multiply(np.array([i.VCDN[k] for k in i.G]),i.probTD[tier]))
            temp2.index=range(1,6)
            Gross_margins["down"+str(tier)]=temp2
            
        dataRT = {None:{
            'RE': dataDA[None]["RE"],
            'CAP': dataDA[None]["CAP"],
            'VC': dataDA[None]["VC"],
            'VCUP': dataDA[None]["VCUP"],
            'VCDN': dataDA[None]["VCDN"],
            'DEMAND': dataDA[None]["DEMAND"],
            'D1': dataDA[None]["D1"],
            'D2': dataDA[None]["D2"],
            'prob':{1: 0.2, 2:0.2, 3:0.2, 4:0.2, 5:0.2},
              'RR': dataDA[None]["RR"],
            'xDA':Energy['en'],
             'PEN':dataDA[None]["PEN"],
             'PENDN':dataDA[None]["PENDN"],
             'REDA':{None: temp},
             'DAdr':{None: i.d.value}
            }}
        iRT = RTsim.create_instance(dataRT)
        opt.solve(iRT)
        RTmargins=pd.DataFrame()
        for s in iRT.S:  
            RTmargins[s]=(np.multiply(iRT.dual[iRT.Con3[s]]-iRT.prob[s]*np.array([iRT.VC[k] for k in iRT.G]),np.array([iRT.xup[s,k].value-iRT.xdn[s,k].value  for k in i.G])))
        RTmargins.index=range(1,len(RTmargins)+1)
        RTpayoffs=pd.DataFrame()
        for s in iRT.S: 
            if s<5:
                RTpayoffs["UP"+str(s)]=-(iRT.dual[iRT.Con3[s]]*df[["hsu","G"]].loc[df["R"]>=s].groupby("G").sum().values.transpose()-np.multiply(iRT.prob[s]*np.array([iRT.VCUP[k] for k in iRT.G]),df[["hsu","G"]].loc[df["R"]>=s].groupby("G").sum().values.transpose()))[0]
            if s>1:
                 RTpayoffs["DN"+str(s)]=(iRT.dual[iRT.Con3[s]]*df[["hsd","G"]].loc[df["R"]<s].groupby("G").sum().values.transpose()-np.multiply(iRT.prob[s]*np.array([iRT.VCDN[k] for k in iRT.G]),df[["hsd","G"]].loc[df["R"]<s].groupby("G").sum().values.transpose()))[0]
        RTpayoffs.index=range(1,len(RTpayoffs)+1)
        for s in i.S:
           Total.at['cost',s]=iRT.prob[s]*(sum(np.multiply(np.array([i.VCUP[k] for k in i.G]),np.array([iRT.xup[s,g].value for g in iRT.G])))+sum(np.multiply(np.array([i.VCDN[k] for k in i.G]),np.array([-iRT.xdn[s,g].value for g in iRT.G]))))
           Total.at['price',s]=iRT.dual[iRT.Con3[s]]
           Total.at['unmet_demand',s]=iRT.prob[s]*(iRT.D1*iRT.d[s].value+iRT.D2*iRT.d[s].value*iRT.d[s].value)
           Total.at['curtail cost',s]=iRT.prob[s]*(iRT.PENDN*iRT.sdup[s].value)
        price_convergence=Total.at['price','DA']-Total[Total.columns[1:]].loc['price'].sum()
        premium_convergence=pd.DataFrame()
        premiums=Gross_margins[Gross_margins.columns[Gross_margins.columns.str.startswith('up')]].sum().sum()+Gross_margins[Gross_margins.columns[Gross_margins.columns.str.startswith('down')]].sum().sum()
        premium_convergence['UP']=Gross_margins[Gross_margins.columns[Gross_margins.columns.str.startswith('up')]].sum(axis=1)+RTpayoffs[RTpayoffs.columns[RTpayoffs.columns.str.startswith('UP')]].sum(axis=1)
        premium_convergence['DN']=Gross_margins[Gross_margins.columns[Gross_margins.columns.str.startswith('down')]].sum(axis=1)+RTpayoffs[RTpayoffs.columns[RTpayoffs.columns.str.startswith('DN')]].sum(axis=1)
        premium_convergence.index.name='Generator'
        premium_convergence.index=premium_convergence.index+1
        Total_margin=pd.DataFrame()
        for k in range(1,6):
             Total_margin[k]=iRT.prob[k]*Gross_margins.sum(axis=1)+RTmargins[k]+RTpayoffs[RTpayoffs.columns[RTpayoffs.columns.str.endswith(str(k))]].sum(axis=1)
             Total_margin.at['RE',k]=(iRT.dual[iRT.Con3[k]]*(iRT.rgup[k].value-iRT.rgdn[k].value)) +iRT.prob[k]*temp*i.dual[i.Con3]-iRT.prob[k]*premiums
             if k>1:
                Total_margin.at['RE',k]=Total_margin.at['RE',k]-RTpayoffs["DN"+str(k)].sum()
             if k<5:
                Total_margin.at['RE',k]=Total_margin.at['RE',k]-RTpayoffs["UP"+str(k)].sum()
             Total_margin.at['DR',k]=iRT.dual[iRT.Con3[k]]*iRT.d[k].value+iRT.prob[k]*i.d.value*i.dual[i.Con3] -iRT.prob[k]*(i.D1*(iRT.d[k].value+i.d.value))-iRT.prob[k]*(i.D2*(iRT.d[k].value+i.d.value)*(iRT.d[k].value+i.d.value))
        # writer = pd.ExcelWriter('Test_output_files/Results_SectionV/FO_EXP_'+l+"_v"+date.today().strftime("%Y_%m_%d")+'revision.xlsx')      
        # round(premium_convergence,2).to_excel(writer,'Premium_convergence')
        # round(Total,2).to_excel(writer,'System_metrics')
        # round(RTmargins,2).to_excel(writer,'RTmargins')
        # round(RTpayoffs,2).to_excel(writer,'RTpayoffs')
        # round(Gross_margins,2).to_excel(writer,'Grossmargins')
        # round(df,2).to_excel(writer,'FO_supply_AWARDS')
        # round(demand,2).to_excel(writer,'FO_demand_AWARDS')
        # round(Energy,2).to_excel(writer,'DA_energy')
        # round(Total_margin,2).to_excel(writer,'Totalmargins')
        # round(Prices,2).to_excel(writer,'Prices')
        # pd.DataFrame(dataDA).to_excel(writer,'input_DA')
        # pd.DataFrame(dataRT).to_excel(writer,'input_RT')
        # writer.close()
        Summary_cost[l]=Total.loc['cost'].transpose()
        Summary_costD[l]=Total.loc['unmet_demand'].transpose()
        Summary_price_convergence[l]=Total.loc['price'].transpose()
        Summary_premium_convergence_up[l]=premium_convergence['UP']
        Summary_premium_convergence_down[l]=premium_convergence['DN']
        #Summary_margin_intermittency[l]=Total_margin.apply(lambda row: sum(row>0.01),axis=1)
        Summary_costCURT[l]=Total.loc['curtail cost'].transpose()


3 RangeSet Declarations
    G : Dimen=1, Size=5, Bounds=(1, 5)
        Key  : Finite : Members
        None :   True :   [1:5]
    R : Dimen=1, Size=4, Bounds=(1, 4)
        Key  : Finite : Members
        None :   True :   [1:4]
    S : Dimen=1, Size=5, Bounds=(1, 5)
        Key  : Finite : Members
        None :   True :   [1:5]

15 Param Declarations
    CAP : Size=5, Index=G, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :    50
          2 :    10
          3 :    10
          4 :    10
          5 :    10
    D1 : Size=1, Index=None, Domain=NonNegativeIntegers, Default=None, Mutable=False
        Key  : Value
        None :     5
    D2 : Size=1, Index=None, Domain=NonNegativeIntegers, Default=None, Mutable=False
        Key  : Value
        None : 550.0
    DEMAND : Size=1, Index=None, Domain=NonNegativeIntegers, Default=None, Mutable=False
        Key  : Value
        None :   200
    PEN : Size=1, Index=None, Domain=NonNegativeIntegers, Default=None, 